In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mlflows-artifacts-remote/1', creation_time=1718307172999, experiment_id='1', last_update_time=1718307172999, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('../../../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../../../data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [6]:
with mlflow.start_run() as run:
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    run_id = run.info.run_id
    print(f"Run ID: {run_id}")

    # dv = DictVectorizer()
    # model = RandomForestRegressor(**params, n_jobs=-1)

    # X_train = dv.fit_transform(dict_train)
    # model.fit(X_train, y_train)

    # X_val = dv.transform(dict_val)
    # y_pred = model.predict(X_val)

    pipeline = make_pipeline(
       DictVectorizer(),
       RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    # mlflow.sklearn.log_model(model, artifact_path="model")

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

    # with open('dict_vectorizer.bin','wb') as f_out:
    #     pickle.dump(dv,f_out)

    # mlflow.log_artifact('dict_vectorizer.bin')

   

Run ID: 95182c88ec8040888af37b5f0259e733


/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [7]:
from mlflow.tracking import MlflowClient


In [17]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflows-artifacts-remote/1', creation_time=1718307172999, experiment_id='1', last_update_time=1718307172999, lifecycle_stage='active', name='green-taxi-duration', tags={}>,
 <Experiment: artifact_location='s3://mlflows-artifacts-remote/0', creation_time=1718307001186, experiment_id='0', last_update_time=1718307001186, lifecycle_stage='active', name='Default', tags={}>]

In [18]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '9a453d9f8fd04d18a65d2da60955675b'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [19]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()

In [20]:
logged_model = f'runs:/{RUN_ID}/model'

In [21]:
dependencies = mlflow.pyfunc.get_model_dependencies(logged_model)
print(dependencies)

2024/06/13 20:23:00 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /tmp/tmp_l3wg7j5/model/requirements.txt'.


/tmp/tmp_l3wg7j5/model/requirements.txt
